In [74]:
import os
import sys
from urllib.parse import urlparse
from IPython.core.getipython import get_ipython
sys.path.append(os.path.join(os.getcwd(), get_ipython().starting_dir))
from app.scraper.scraper import Scraper
from app.utils import get_chrome_driver, get_website_html
from dotenv import load_dotenv
load_dotenv()    
import openai
import pandas as pd
import backoff
import html2text
pd.set_option('display.max_colwidth', None)


## Helper functions

In [47]:

def get_file_type(url):
    if url.endswith(".pdf"):
        return "pdf"
    elif url.endswith(".docx"):
        return "docx"
    elif url.endswith(".doc"):
        return "doc"
    elif url.endswith(".zip"):
        return "zip"
    elif url.endswith(".xlsx"):
        return "xlsx"
    elif url.endswith(".xls"):
        return "xls"
    elif "fileadmin" in urlparse(url).path:
        return "file"
    else:
        return "website"

@backoff.on_exception(backoff.expo, openai.RateLimitError)
def completions_with_backoff(**kwargs):
    return openai.completions.create(**kwargs)


def is_file_law_or_regulation(df):

    prompt_template = (
        "Based on the name of the file, does the data contain text of a Law or Regulation? "
        "Answer ONLY with one of: ['yes', 'no', 'unclear']. Filename: {filename}"
    )

    outputs = []
    current_batch = []
    current_batch_len = 0
    max_prompt_length = 10000
    max_parallel_prompts = 20  # defined by OpenAI limits
    TOTAL_TOKENS_CONSUMED = 0
    for idx, filename in enumerate(df["details_href"]):
        if idx % 100 == 0:
            print(f"Processing filename {idx}/{len(df)}")
        current_prompt = prompt_template.format(filename=filename)
        if current_batch_len + len(current_prompt) > max_prompt_length or idx >= len(df) -1 or len(current_batch) == max_parallel_prompts -1:
    
            # We can still add one prompt because we are easily below the limit of the API
            current_batch.append(current_prompt)
            current_batch_len += len(current_prompt)

            # We have a full batch --> send the request to openAI and process the results
            response = completions_with_backoff(
                model="gpt-3.5-turbo-instruct",
                prompt=current_batch,
                max_tokens=5,
            )

            # match completions to prompts by index
            current_batch_outputs = [""] * len(current_batch)
            for choice in response.choices:
                current_batch_outputs[choice.index] = choice.text.lower().strip()
            TOTAL_TOKENS_CONSUMED += response.usage.total_tokens
            # Add the current batch outpus to the outputs
            outputs.extend(current_batch_outputs)
            current_batch = []
            current_batch_len = 0
        else:
            current_batch.append(current_prompt)
            current_batch_len += len(current_prompt)

    print("Total tokens consumed: ", TOTAL_TOKENS_CONSUMED)
    df["is_law_or_regulation"] = outputs

    # Ensure the is_law_regulation column is only one of the three values
    df["is_law_or_regulation"] = df["is_law_or_regulation"].apply(clean_is_law_or_regulation_value)
    return df

def clean_is_law_or_regulation_value(x):
    if "yes" in x.lower() and "no" not in x.lower() and "unclear" not in x.lower():
        return "yes"
    elif "no" in x.lower() and "yes" not in x.lower() and "unclear" not in x.lower():
        return "no"
    elif "unclear" in x.lower() and "yes" not in x.lower() and "no" not in x.lower():
        return "unclear"
    else:
        return "unclear"



In [48]:
METADATA_DIR = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/"
RAW_DATA_DIR = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/testing_integration"
scraper = Scraper(os.path.join(METADATA_DIR, "references.csv"), RAW_DATA_DIR, local=True)

### Objective

I want to scrape the references that actually link to a regulation or a law.

Steps:

1. check if reference is a file or a website based on the filename
2. if it is a file, check if it contains a law or regulation based on its filename
3. if it is a website, access the website and check if it contains a law or regulation
4. create a processed table with only the references that are law or regulation and with their metadata


In [49]:
# STEP 1 - check if reference is a file or a website based on the filename
import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

data = scraper.references_data
data = data[data["details_href"].notnull()]  # only keep rows that have an actual details href
data["base_url"] = data["details_href"].apply(lambda x: urlparse(x).netloc)
data["is_processed"] = data["is_processed"].replace({"True": 1, "False": 0}).astype(float).astype(bool)
data["file_type"] = data["details_href"].apply(get_file_type)
data_websites = data[data["file_type"] == "website"]
data_files = data[data["file_type"] != "website"]

In [55]:
# STEP 2 - if it is a file, check if it contains a law or regulation based on its filename
if os.path.exists("data_file_is_law_or_regulation_list.txt"):
    with open("data_file_is_law_or_regulation_list.txt", "r") as file:
        law_or_regulation_data = file.readlines()
        law_or_regulation_data = [line.strip() for line in law_or_regulation_data]
    data_files["is_law_or_regulation"] = law_or_regulation_data
else:
    data_files = is_file_law_or_regulation(data_files)
    is_law_or_regulation_list = data_files["is_law_or_regulation"].tolist()
    with open("data_file_is_law_or_regulation_list.txt", "w") as file:
        for item in is_law_or_regulation_list:
            file.write("%s\n" % item)

print(data_files["is_law_or_regulation"].value_counts())



yes        374
no         219
unclear    174
Name: is_law_or_regulation, dtype: int64


In [79]:
# STEP 3 - check if the website contains a law or regulation

driver = get_chrome_driver(local=True)
for url in data_websites["details_href"]:
    html = get_website_html(url, driver, close_driver=False)
    print(url, len(html), type(html))
    # all_text_elements = html.find_all(text=True)
    # all_text = " ".join(t.strip() for t in all_text_elements)
    # all_text = html.get_text(separator=' ', strip=True)
    # all_text = " ".join([string for string in html.strings])
    # for script in html(["script", "style"]):
    #     script.extract()   
    # all_text = html.get_text()
    # h = html2text.HTML2Text()
    # h.escape_all = True
    # all_text = h.handle(str(html))

    tag = html.find("div", {})
    # Get the whole body tag
    tag = html.body 
    all_text = " ".join([string for string in tag.strings])

    print("Total amount of text: ", len(all_text))
    print(f"All Extracted Text: {all_text}")

    html_str = str(html)
    total_length = len(html_str)
    print(f"Total HTML length: {total_length}")
    # print(html)

    break



http://www.pisrs.si/Pis.web/pregledPredpisa?id=ZAKO6792 1 <class 'bs4.BeautifulSoup'>
Total amount of text:  1157
All Extracted Text: 
  Vstopna stran   ZAKO6792   Zbirke  Išči po celotni vsebini Išči Napredno iskanje Kazalo Oblike zapisa in besedilo  Pomagajte nam izboljšati spletišče:  Ali vam je ta stran koristila?   DA NE KONTAKT Republika Slovenija 
 Služba vlade za zakonodajo 
 Mestni trg 4 
 1000 Ljubljana 
 gp.svz@gov.si 
 pisrs.svz@gov.si DOSTOPNOST Izjava o dostopnosti 
 Dostop za razvijalce  (API PISRS) 
 Priročnik za uporabo logotipa PISRS 
 POVEZAVE Portal GOV.SI 
 eUPRAVA 
 SPOT 
 Na sodišču DRŽAVNE USTANOVE Predsednica republike 
 Državni zbor  /  Državni svet 
 Vlada  /  Ministrstva  /  Vladne službe 
 Predsednik vlade 
 Ustavno sodišče  /  Vrhovno sodišče 
 Računsko sodišče ©2024 Služba Vlade Republike Slovenije za zakonodajo. Vse pravice pridržane! Kazalo strani  |  Pogoji uporabe  |  Več o piškotkih Piškotki in nastavitve zasebnosti  Spletno mesto PISRS uporablja piš

In [81]:
# Let's try to convert a specific file:

path = "/Users/juankostelec/Downloads/testing_conversion.html"
with open(path, "r") as file:
    html = file.read()
    # print(html)
    print(html2text.html2text(html))




**DAVEK NA DODANO VREDNOST**

**Stopnje DDV**

****

****

**9\. izdaja JANUAR 2024**

  
**KAZALO**

****

[1 SPLOŠNO](https://wordtohtml.net/#_Toc155344149)

[2\. PRILOGA I – 9,5 %](https://wordtohtml.net/#_Toc155344150)

[2\. 1 HRANA (VKLJUČNO S PIJAČO, RAZEN ALKOHOLNIH PIJAČ) ZA LJUDI IN ŽIVALI
TER PRIPRAVA JEDI - 1. TOČKA PRILOGE I K
ZDDV-1](https://wordtohtml.net/#_Toc155344151)

[2.1.1 Stopnja DDV za dobavo čebeljega voska in satnih osnov (1. 10.
2019)](https://wordtohtml.net/#_Toc155344152)

[2.1.2 Obračunavanje DDV pri opravljanju gostinske dejavnosti (1. 1.
2020)](https://wordtohtml.net/#_Toc155344153)

[2.1.3 Uvrstitev blaga in stopnja DDV za posušena neoluščena bučna semena
sorte buč Cucurbita pepo L. Cucurbita Maxima (1. 1.
2020)](https://wordtohtml.net/#_Toc155344154)

[2.1.4 Prehranska dopolnila in uporaba nižje stopnje DDV (1. 1.
2020)](https://wordtohtml.net/#_Toc155344155)

[2.1.5 Ali mora davčni zavezanec, ki prodaja blago, pri katerem sta mogoči obe
DDV stopnji, kup

## Overview of the sources which still have files I haven't processed yet

In [10]:

# Let's analyze again what are the base URLK of the websites that were not yet processed
scraper.references_data["is_processed"] = (
scraper.references_data["is_processed"].replace({"True": 1, "False": 0}).astype(float).astype(bool))
unprocessed_references = scraper.references_data[~scraper.references_data["is_processed"]]
unprocessed_references = unprocessed_references[unprocessed_references["details_href"].notnull()]
unprocessed_references["base_url"] = unprocessed_references["details_href"].apply(lambda x: urlparse(x).netloc)
# print(unprocessed_references["base_url"].value_counts())

# Check the reference is a file or website
unprocessed_references["is_file"] = unprocessed_references["details_href"].apply(get_file_type)

unprocessed_files 
print(unprocessed_references["is_file"].value_counts())

# Only consider base URLs with more than 10 references.
relevant_base_urls = unprocessed_references["base_url"].value_counts() > 10
relevant_base_urls = relevant_base_urls[relevant_base_urls].index
unprocessed_references = unprocessed_references[unprocessed_references["base_url"].isin(relevant_base_urls)]




website    532
docx       283
doc        220
pdf        169
zip         58
xlsx        27
file        10
Name: is_file, dtype: int64


In [ ]:
# Check which of the files are laws or regulations. First define a columns determining the type of the
# url. If it is a file, then do the check








## Fu.gov.si analysis


In [30]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

data = unprocessed_references[unprocessed_references["base_url"].str.contains("fu.gov.si")]

# data.describe()

# Count nonzero details href values
unique_details_href = data["details_href"].unique()  # there are 604

# Check what type of file is the details_href
data["file_type"] = data["details_href"].apply(get_file_type)
data["file_type"].value_counts()


# The files I can directly download, what about the websites
website_details_href = data[data["file_type"] == "website"]["details_href"]
clean_website_details_href = website_details_href.apply(lambda x: x.split("#")[0])

# check if the clean websites are acutally in teh reference_href column
print(clean_website_details_href)
clean_website_details_href.isin(data["reference_href"]).sum()


71                                                                                                                                                                                  https://www.fu.gov.si/davki_in_druge_dajatve/podrocja/dohodnina/dohodnina_dohodek_iz_dejavnosti/
110                                                                                                                                                                                 https://www.fu.gov.si/davki_in_druge_dajatve/podrocja/dohodnina/dohodnina_dohodek_iz_dejavnosti/
149                                                                                                                                                                                          https://www.fu.gov.si/davki_in_druge_dajatve/podrocja/dohodnina/letna_odmera_dohodnine/
201                                                                                                                                                                      

/var/folders/y1/f5f_tzds45b44fhpyxx38qs40000gn/T/ipykernel_34515/2418554644.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["file_type"] = data["details_href"].apply(get_file_type)


12